Efficient solution of Poisson's equation using discrete variable representation basis sets for Car-Parrinello *ab initio* molecular dynamics simulations with cluster boundary conditions

Normalized sinc-DVR function ${u_{\alpha}(x)}$
$$
\begin{equation}
u_{\alpha}(x) = \frac{\sqrt{d}}{\pi}
\frac{\sin\left[\pi(x-x_{\alpha})\right]}{x-x_{\alpha}}
\end{equation}
$$
Grid spacing: $d = \pi\hbar/p_{0}$

Hartree potential:
$$
\begin{equation}
V_{\mathrm{H}}(\mathbf{r}) = \int \mathrm{d}\mathbf{r}
\frac{\rho(\mathbf{r}')}{\left|\mathbf{r}'-\mathbf{r}\right|}
\end{equation}
$$

The term $1/r$ is rewritten using the following integral expression for $r^{-\alpha}$:
$$
\begin{equation}
r^{-\alpha} = \frac{2}{\Gamma(\alpha/2)}
\int_{-\infty}^{\infty} e^{-r^2\,e^{2y}+ay}\,\mathrm{d}y
\end{equation}
$$
Using change of variable to $t = e^{y}$, the identity become:
$$
\begin{equation}
r^{-\alpha} = \frac{2}{\Gamma(\alpha/2)}
\int_{-\infty}^{\infty} e^{-r^2 t^{2}}t^{\alpha-1}\,\mathrm{d}y
\end{equation}
$$

Using the identity above, the equation for Hartree potential can be written as:
$$
\begin{equation}
V_{\mathrm{H}}(\mathbf{r}) =
\frac{2}{\sqrt{\pi}}\int_{0}^{\infty}\int\,\mathrm{d}\mathbf{r}'
e^{-t^2\left(\left| \mathbf{r}-\mathbf{r}' \right|^2\right)}
\rho(\mathbf{r}')
\end{equation}
$$

Using direct product of one-dimensional sinc-DVR functions as
$$
\begin{equation}
\rho(\mathbf{r}) = \sum_{\alpha\beta\gamma}
d_{\alpha\beta\gamma} u_{\alpha}(x) u_{\beta}(y) u_{\gamma}(z)
\end{equation}
$$
where $d_{\alpha\beta\gamma}$ is a set of expansion coefficients for $\mathrm{\rho}(\mathbf{r})$